## Parameters

In [1]:
input_data='./input_data/'
output_data='./output_data/'

## Libraries

In [35]:
import numpy as np
import pandas as pd
import datetime

In [55]:
daily=pd.read_csv(output_data + 'ngonye_flow_daily.csv')
daily=daily.drop(['LaggedDate','VicFalls','Conversion','Flow_difference','Volume','MonthId'],axis=1)
daily=daily.set_index(pd.DatetimeIndex(daily['Date']))
daily.head(4)

,Date,Flow,Exceedance,Year,Month,Day,WaterYear,WaterMonth,WaterDay,WaterWeek
Date,,,,,,,,,,
1924-10-01,1924-10-01,111.133124,0.999,1924,10,1,1924,1,1,1
1924-10-02,1924-10-02,111.133124,0.999,1924,10,2,1924,1,2,1
1924-10-03,1924-10-03,111.133124,0.999,1924,10,3,1924,1,3,1
1924-10-04,1924-10-04,111.133124,0.999,1924,10,4,1924,1,4,1


Load the worst case impact table. 

WC1 and WC2 are the two worst case impact predictions giving reduction in flow by month for 2070.
- *WC1*: Hamududu & Killingtviet 2016
- *WC2*: Harrison & Whittington, 2002

In [44]:
cc_cases=pd.read_csv(input_data + 'climate_change_worst_cases.csv').set_index('Month')
cc_cases

,WC1,WC2
Month,,
10,0.16,0.24
11,0.20,0.30
12,0.17,0.40
1,0.18,0.33
2,0.16,0.32
3,0.41,0.40
4,0.29,0.39
5,0.24,0.36
6,0.24,0.35


Calculate the Climate Change Rate which is 0% in 1990 and rises to 100% in 2070. This sets the proportion of the overall climate change impact (by 2070) which is assumed to apply to any given date.

In [91]:
daily['CCRate']=daily.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
daily

,Date,Flow,Exceedance,Year,Month,Day,WaterYear,WaterMonth,WaterDay,WaterWeek,CCRate
Date,,,,,,,,,,,
1924-10-01,1924-10-01,111.133124,0.999,1924,10,1,1924,1,1,1,0.0000
1924-10-02,1924-10-02,111.133124,0.999,1924,10,2,1924,1,2,1,0.0000
1924-10-03,1924-10-03,111.133124,0.999,1924,10,3,1924,1,3,1,0.0000
1924-10-04,1924-10-04,111.133124,0.999,1924,10,4,1924,1,4,1,0.0000
1924-10-05,1924-10-05,111.133124,0.999,1924,10,5,1924,1,5,1,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
2017-09-26,2017-09-26,203.017496,0.950,2017,9,26,2016,12,361,52,0.3375
2017-09-27,2017-09-27,203.017496,0.950,2017,9,27,2016,12,362,52,0.3375
2017-09-28,2017-09-28,203.017496,0.950,2017,9,28,2016,12,363,52,0.3375


In [46]:
daily_future=pd.DataFrame(index=pd.date_range(start='10/1/2017', end='09/30/2053', freq='D'))
daily_future['Year']=daily_future.index.year
daily_future['CCRate']=daily_future.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
daily_future

,Year,CCRate
2017-10-01,2017,0.3375
2017-10-02,2017,0.3375
2017-10-03,2017,0.3375
2017-10-04,2017,0.3375
2017-10-05,2017,0.3375
...,...,...
2053-09-26,2053,0.7875
2053-09-27,2053,0.7875
2053-09-28,2053,0.7875
2053-09-29,2053,0.7875


In [76]:
start_year=1933
start_date=datetime.date(start_year, 10, 1)
start_date

datetime.date(1933, 10, 1)

In [90]:
daily_future['SampleDate']=pd.date_range(start=start_date, periods=13149, freq='D')
daily_future

,Year,CCRate,SampleDate,ExistingCCRate
2017-10-01,2017,0.3375,1933-10-01,0.3375
2017-10-02,2017,0.3375,1933-10-02,0.3375
2017-10-03,2017,0.3375,1933-10-03,0.3375
2017-10-04,2017,0.3375,1933-10-04,0.3375
2017-10-05,2017,0.3375,1933-10-05,0.3375
...,...,...,...,...
2053-09-26,2053,0.7875,1969-09-26,0.7875
2053-09-27,2053,0.7875,1969-09-27,0.7875
2053-09-28,2053,0.7875,1969-09-28,0.7875
2053-09-29,2053,0.7875,1969-09-29,0.7875


In [93]:
daily_future['ExistingCCRate']=daily_future.join(daily,on='SampleDate',rsuffix='_daily')['CCRate_daily']
daily_future

,Year,CCRate,SampleDate,ExistingCCRate
2017-10-01,2017,0.3375,1933-10-01,0.0
2017-10-02,2017,0.3375,1933-10-02,0.0
2017-10-03,2017,0.3375,1933-10-03,0.0
2017-10-04,2017,0.3375,1933-10-04,0.0
2017-10-05,2017,0.3375,1933-10-05,0.0
...,...,...,...,...
2053-09-26,2053,0.7875,1969-09-26,0.0
2053-09-27,2053,0.7875,1969-09-27,0.0
2053-09-28,2053,0.7875,1969-09-28,0.0
2053-09-29,2053,0.7875,1969-09-29,0.0
